# RAG QA Arena Benchmark: Complete Evaluation Pipeline

This notebook provides a comprehensive pipeline for evaluating Retrieval-Augmented Generation (RAG) systems using the [RAG QA Arena benchmark](https://github.com/awslabs/rag-qa-arena).

## What The Notebook Does
- How to ingest 30GB+ document collections into Contextual AI Datastore
- Performance optimization for large-scale document processing  
- Retrieval evaluation metrics and answer equivalence testing
- Production-ready error handling and recovery procedures

**Datasets**: LOTTE (~150K docs) and FIQA (~57K docs)

## Setup and Imports

We import all necessary libraries for data handling, evaluation, and asynchronous operations.  
The notebook uses `pandas` for dataframes, `numpy` for numerical operations, and `ragas` for retrieval metrics.

In [1]:
# Data processing
import json
import pandas as pd
import numpy as np

# Async operations and file handling
import asyncio
import tempfile
import os
import time
import csv
from typing import Dict, List, Tuple

# Dataset loading
from datasets import load_dataset

# RAG evaluation
from ragas import evaluate
from ragas.metrics import context_recall, context_precision, faithfulness

# Contextual AI client
from contextual import ContextualAI

## 📊 Dataset Overview

| Dataset | Size | Documents | Domain |
|---------|------|-----------|--------|
| LOTTE | 30GB | 150K+ | Lifestyle/Recreation |
| FIQA | - | 57K | Financial QA |

### Download Instructions

## Load Data

To replicate the RAG QA Benchmark we need two datasets for the corpus, LOTTE and FIQA, and the annotoated question/answers which are available at https://github.com/awslabs/rag-qa-arena/tree/main/data

The bulk of this notebook focuses on showing how to load LOTTE and FIQA into a Contextual AI Datastore

## Download Lotte Passages

Lotte is about 30GBs so be prepared

I did this:
```
# Clone the repo
git clone https://huggingface.co/datasets/colbertv2/lotte_passages

# Navigate into the repo
cd lotte_passages

# Pull the large files
git lfs pull

Let's look at the dataset

In [ ]:
!head -n 2 /Users/rajivshah/Code/lotte_passages/recreation/dev_collection.jsonl

In [ ]:
# looking up a random line
!sed -n '72610p' /Users/rajivshah/Code/lotte_passages/recreation/test_collection.jsonl

In [ ]:
## to check the number of lines in the file
!wc -l /Users/rajivshah/Code/lotte_passages/recreation/test_collection.jsonl

## Download FIQA

You can get FIQA from https://huggingface.co/datasets/BeIR/fiqa


In [ ]:
from datasets import load_dataset
ds = load_dataset("BeIR/fiqa", "corpus")

In [ ]:
ds['corpus'][42]  

##  Annotated Questions and Answers:

From: https://github.com/awslabs/rag-qa-arena

In [ ]:
dataset = load_dataset("rajistics/rag-qa-arena")

Check out Lotte recreation

In [ ]:
recreation_data = dataset['recreation']
print(recreation_data[:2])

In [ ]:
#!head -n 2 /Users/rajivshah/Code/rag-qa-arena/data/annotations_recreation_with_citation.jsonl

In [ ]:
!sed -n '72610p' /Users/rajivshah/Code/lotte_passages/recreation/test_collection.jsonl

In [ ]:
#!tail -n 2 /Users/rajivshah/Code/rag-qa-arena/data/annotations_recreation_with_citation.jsonl

In [ ]:
!sed -n '152852p' /Users/rajivshah/Code/lotte_passages/recreation/test_collection.jsonl

Check out FIQA dataset

In [ ]:
#!head -n 2 /Users/rajivshah/Code/rag-qa-arena/data/annotations_fiqa_with_citation.jsonl
fiqa_data = dataset['fiqa']
print(fiqa_data[1:2])

In [ ]:
ds['corpus'][42]

In [ ]:
result = ds['corpus'].filter(lambda x: x['_id'] == '382384')
print(result[0])

### Ingest into Contextual AI Datastore

In [34]:
from contextual import ContextualAI
client = ContextualAI()  ## use your API key if you already set

In [ ]:

result = client.datastores.create(name="RAGQA_Recreation2")
datastore_id = result.id
print(f"Datastore ID: {datastore_id}")

# JSONL Document Ingestion Script Documentation

## Overview

This script processes large JSONL (JSON Lines) files containing document collections and uploads them to a datastore while maintaining precise tracking of document relationships. It's specifically designed for handling large document collections (like the 150K+ document LOTTE passages collection) with controlled concurrency to prevent system overload.

## Key Features

### Document Processing
- **JSONL Parsing**: Reads files where each line contains a JSON document with fields like `doc_id`, `author`, and `text`
- **Markdown Formatting**: Converts each document to text with metadata headers and content sections
- **Meaningful Filenames**: Creates temporary files named `doc_{document_id}.md` instead of random temporary names

### Concurrency Control
- **Throttled Uploads**: Limits concurrent processing to prevent overwhelming the datastore (default: 4 simultaneous documents)
- **Smart Waiting**: Monitors document status and waits for available processing slots before uploading new documents
- **Status Monitoring**: Regularly checks and reports on document processing states (pending, processing, processed)

### Comprehensive Tracking
- **Line Number Mapping**: Maintains exact relationship between original JSONL line numbers and datastore document IDs
- **CSV Logging**: Creates persistent `document_mapping.csv` file with complete upload history
- **Progress Monitoring**: Shows real-time processing rates and status breakdowns

## How It Works

### 1. Document Reading and Parsing
```
JSONL File (line 152852) → Parse JSON → Extract doc_id, author, text
```

### 2. Markdown Conversion
Each document is formatted as:
```markdown
# Document {doc_id}

**Author:** {author}
**Original Doc ID:** {doc_id}
**Line Number:** {line_number}

---

## Content

{formatted_text}
```

### 3. Concurrency Management
- Before each upload, checks current processing queue
- If queue is full (≥4 documents), waits and checks every 30 seconds
- Only proceeds when a processing slot becomes available

### 4. Upload and Tracking
- Uploads document to datastore via API
- Records mapping: `line_number → original_doc_id → datastore_document_id`
- Logs to CSV file for permanent tracking
- Displays progress statistics

### 5. Status Monitoring
- Shows processing rates (documents/minute)
- Provides status breakdowns every few uploads
- Final status summary at completion

## Output Files

### document_mapping.csv
Permanent tracking file with columns:
- `line_number`: Original position in JSONL file
- `original_doc_id`: Document ID from JSON data
- `datastore_document_id`: New ID assigned by datastore
- `author`: Document author
- `text_preview`: First 100 characters for identification

### Temporary Files
- Created as `doc_{document_id}.txt` in system temp directory
- Automatically cleaned up after upload
- Visible in datastore interface during processing

## Configuration Options

- `max_documents`: Limit processing to first N documents (useful for testing)
- `max_concurrent`: Maximum simultaneous processing documents (default: 4)
- `check_interval`: How often to check status when waiting (default: 30 seconds)
- `mapping_file`: Name of CSV tracking file (default: "document_mapping.csv")

## Usage Patterns

### Testing (10 documents)
```python
mappings = ingest_documents_with_tracking(
    jsonl_file_path="test_collection.jsonl",
    client=client,
    datastore_id=datastore_id,
    max_documents=10
)
```

### Production (with rate limiting)
```python
mappings = ingest_documents_with_tracking(
    jsonl_file_path="test_collection.jsonl",
    client=client,
    datastore_id=datastore_id,
    max_documents=1000,
    max_concurrent=4,
    check_interval=30
)
```

## Utility Functions

- `load_mapping_from_file()`: Read existing mapping data from CSV
- `find_document_by_line_number()`: Look up document info by original line number
- `get_active_document_counts()`: Check current processing queue status
- `analyze_document_status()`: Display comprehensive status breakdown

## Benefits

1. **Traceability**: Always know which datastore document corresponds to which original line
2. **Reliability**: Handles failures gracefully, continues processing remaining documents
3. **Efficiency**: Respects system limits while maintaining good throughput
4. **Resumability**: CSV tracking allows resuming from failures or adding more documents later
5. **Visibility**: Clear progress reporting and status monitoring throughout the process

In [77]:
import json
import csv
import tempfile
import os
import time
from typing import Dict, List, Tuple

def get_active_document_counts(client, datastore_id: str) -> tuple:
    """
    Get count of documents currently being processed or pending.
    
    Args:
        client: API client instance
        datastore_id: ID of the datastore
    
    Returns:
        Tuple of (processing count, pending count)
    """
    response = client.datastores.documents.list(datastore_id)
    processing = 0
    pending = 0
    
    for doc in response.documents:
        if doc.status == 'processing':
            processing += 1
        elif doc.status == 'pending':
            pending += 1
            
    return processing, pending

def wait_for_available_slot(client, datastore_id: str, max_concurrent: int = 4, 
                        check_interval: int = 30) -> None:
    """
    Wait until there's a free slot for processing or pending.
    
    Args:
        client: API client instance
        datastore_id: ID of the datastore
        max_concurrent: Maximum number of concurrent documents (processing or pending)
        check_interval: How often to check status in seconds
    """
    while True:
        processing_count, pending_count = get_active_document_counts(client, datastore_id)
        total_active = processing_count + pending_count
        
        if total_active < max_concurrent:
            return
            
        print(f"Currently {processing_count} processing and {pending_count} pending documents.")
        print(f"Waiting {check_interval} seconds for a slot to become available...")
        time.sleep(check_interval)

def analyze_document_status(client, datastore_id: str):
    """
    Analyze the status of documents in the datastore
    
    Args:
        client: API client instance
        datastore_id: ID of the datastore
    """
    response = client.datastores.documents.list(datastore_id)
    status_counts = {}
    
    for doc in response.documents:
        status = doc.status
        status_counts[status] = status_counts.get(status, 0) + 1
    
    print(f"\nDocument Status Analysis")
    print(f"----------------------")
    print(f"Total Documents in System: {len(response.documents)}")
    print("\nStatus Breakdown:")
    for status, count in status_counts.items():
        print(f"- {status}: {count} documents")

def ingest_documents_with_tracking(
    jsonl_file_path: str, 
    client, 
    datastore_id: str, 
    max_documents: int = 10,
    max_concurrent: int = 4,
    check_interval: int = 30,
    mapping_file: str = "document_mapping.csv"
) -> List[Tuple[int, str, str]]:
    """
    Ingest documents from JSONL file and track line number to document ID mapping.
    
    Args:
        jsonl_file_path: Path to the JSONL file
        client: Your datastore client
        datastore_id: Target datastore ID
        max_documents: Maximum number of documents to process
        max_concurrent: Maximum number of documents processing/pending at once
        check_interval: How often to check status when waiting (seconds)
        mapping_file: CSV file to store the mapping
        
    Returns:
        List of tuples (line_number, doc_id, document_id)
    """
    
    mappings = []
    processed_count = 0
    start_time = time.time()
    
    # Initialize or append to mapping file
    file_exists = os.path.exists(mapping_file)
    
    with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
        with open(mapping_file, 'a', newline='') as csv_file:
            writer = csv.writer(csv_file)
            
            # Write header if file is new
            if not file_exists:
                writer.writerow(['line_number', 'original_doc_id', 'datastore_document_id', 'author', 'text_preview'])
            
            for line_number, line in enumerate(jsonl_file, 1):
                if processed_count >= max_documents:
                    break
                    
                try:
                    # Parse the JSON line
                    doc_data = json.loads(line.strip())
                    
                    # Extract document information
                    original_doc_id = doc_data.get('doc_id', 'unknown')
                    author = doc_data.get('author', 'unknown')
                    text = doc_data.get('text', '')
                    text_preview = text[:100] + "..." if len(text) > 100 else text
                    
                    # Wait for an available slot before processing
                    print(f"\nPreparing to process line {line_number} (Doc ID: {original_doc_id})")
                    wait_for_available_slot(client, datastore_id, max_concurrent, check_interval)
                    
                    # Create a temporary markdown file with the document content using the document ID
                    # Sanitize the document ID for filename use
                    safe_doc_id = str(original_doc_id).replace('/', '_').replace('\\', '_')
                    temp_filename = f"doc_{safe_doc_id}.txt"
                    temp_file_path = os.path.join(tempfile.gettempdir(), temp_filename)
                    
                    with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
                        # Write document metadata and content in markdown format
                        temp_file.write(f"# Document {original_doc_id}\n\n")
                        #temp_file.write(f"**Author:** {author}  \n")
                        temp_file.write(f"**Original Doc ID:** {original_doc_id}  \n")
                        temp_file.write(f"**Line Number:** {line_number}  \n\n")
                        temp_file.write("---\n\n")
                        temp_file.write(f"## Content\n\n")
                        
                        # Format the main content - you could add paragraph breaks or other formatting here
                        formatted_text = text.replace('\n\n', '\n\n')  # Ensure proper paragraph spacing
                        temp_file.write(formatted_text)
                        # temp_file_path is already set above
                    
                    try:
                        # Ingest the document
                        with open(temp_file_path, 'rb') as f:
                            ingestion_result = client.datastores.documents.ingest(datastore_id, file=f)
                            datastore_document_id = ingestion_result.id
                        
                        # Record the mapping
                        mapping_entry = (line_number, original_doc_id, datastore_document_id)
                        mappings.append(mapping_entry)
                        
                        # Write to CSV
                        writer.writerow([line_number, original_doc_id, datastore_document_id, author, text_preview])
                        
                        processed_count += 1
                        elapsed_time = (time.time() - start_time) / 60  # in minutes
                        rate = processed_count / elapsed_time if elapsed_time > 0 else 0
                        
                        print(f"✓ Line {line_number}: Original ID {original_doc_id} -> Datastore ID {datastore_document_id}")
                        print(f"   Processing rate: {rate:.1f} documents/minute")
                        
                        # Show current status every few documents
                        if processed_count % 3 == 0:
                            analyze_document_status(client, datastore_id)
                        
                    finally:
                        # Clean up temporary file
                        os.unlink(temp_file_path)
                        
                except json.JSONDecodeError as e:
                    print(f"✗ Line {line_number}: JSON parsing error - {e}")
                    continue
                except Exception as e:
                    print(f"✗ Line {line_number}: Ingestion error - {e}")
                    continue
    
    print(f"\nProcessed {processed_count} documents successfully")
    print(f"Mapping saved to: {mapping_file}")
    
    # Final status check
    print(f"\nFinal status check:")
    analyze_document_status(client, datastore_id)
    
    return mappings

def load_mapping_from_file(mapping_file: str = "document_mapping.csv") -> Dict[int, Dict[str, str]]:
    """
    Load the line number to document ID mapping from CSV file.
    
    Returns:
        Dictionary mapping line_number to document info
    """
    mapping = {}
    
    if not os.path.exists(mapping_file):
        print(f"Mapping file {mapping_file} not found")
        return mapping
    
    with open(mapping_file, 'r', newline='') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            line_num = int(row['line_number'])
            mapping[line_num] = {
                'original_doc_id': row['original_doc_id'],
                'datastore_document_id': row['datastore_document_id'],
                'author': row['author'],
                'text_preview': row['text_preview']
            }
    
    return mapping

def find_document_by_line_number(line_number: int, mapping_file: str = "document_mapping.csv") -> Dict[str, str]:
    """
    Find document information by line number.
    """
    mapping = load_mapping_from_file(mapping_file)
    return mapping.get(line_number, {})


In [ ]:
# Replace these with your actual values
JSONL_FILE_PATH = "/Users/rajivshah/Code/lotte_passages/recreation/test_collection.jsonl"
# CLIENT = your_client_instance
# DATASTORE_ID = "your_datastore_id"

print("Document Ingestion Test Script")
print("=" * 40)

mappings = ingest_documents_with_tracking(
    jsonl_file_path=JSONL_FILE_PATH,
    client=client,
    datastore_id=datastore_id,
    max_documents=100,
    max_concurrent=10,  # Only allow 4 documents processing/pending at once
    check_interval=20  # Check every 30 seconds when waiting
)

# For now, just show what the mapping file would look like
print("This script will create a CSV mapping file with columns:")
print("- line_number: Original line number from JSONL file")
print("- original_doc_id: The 'doc_id' field from the JSON")
print("- datastore_document_id: The ID returned by the datastore")
print("- author: Document author")
print("- text_preview: First 100 characters of text")

### Ingest FIQA a Hugging Face dataset

This script is modified from above for a Hugging Face dataset

In [76]:
import csv
import tempfile
import os
import time
from typing import Dict, List, Tuple
from datasets import load_dataset

def get_active_document_counts(client, datastore_id: str) -> tuple:
    """
    Get count of documents currently being processed or pending.
    
    Args:
        client: API client instance
        datastore_id: ID of the datastore
    
    Returns:
        Tuple of (processing count, pending count)
    """
    response = client.datastores.documents.list(datastore_id)
    processing = 0
    pending = 0
    
    for doc in response.documents:
        if doc.status == 'processing':
            processing += 1
        elif doc.status == 'pending':
            pending += 1
            
    return processing, pending

def wait_for_available_slot(client, datastore_id: str, max_concurrent: int = 4, 
                        check_interval: int = 30) -> None:
    """
    Wait until there's a free slot for processing or pending.
    
    Args:
        client: API client instance
        datastore_id: ID of the datastore
        max_concurrent: Maximum number of concurrent documents (processing or pending)
        check_interval: How often to check status in seconds
    """
    while True:
        processing_count, pending_count = get_active_document_counts(client, datastore_id)
        total_active = processing_count + pending_count
        
        if total_active < max_concurrent:
            return
            
        print(f"Currently {processing_count} processing and {pending_count} pending documents.")
        print(f"Waiting {check_interval} seconds for a slot to become available...")
        time.sleep(check_interval)

def analyze_document_status(client, datastore_id: str):
    """
    Analyze the status of documents in the datastore
    
    Args:
        client: API client instance
        datastore_id: ID of the datastore
    """
    response = client.datastores.documents.list(datastore_id)
    status_counts = {}
    
    for doc in response.documents:
        status = doc.status
        status_counts[status] = status_counts.get(status, 0) + 1
    
    print(f"\nDocument Status Analysis")
    print(f"----------------------")
    print(f"Total Documents in System: {len(response.documents)}")
    print("\nStatus Breakdown:")
    for status, count in status_counts.items():
        print(f"- {status}: {count} documents")

def ingest_hf_dataset_with_tracking(
    dataset,
    client, 
    datastore_id: str, 
    max_documents: int = 10,
    max_concurrent: int = 4,
    check_interval: int = 30,
    mapping_file: str = "hf_document_mapping.csv"
) -> List[Tuple[int, str, str]]:
    """
    Ingest documents from an already-loaded Hugging Face dataset and track row index to document ID mapping.
    
    Args:
        dataset: Already loaded Hugging Face dataset (e.g., ds['corpus'])
        client: Your datastore client
        datastore_id: Target datastore ID
        max_documents: Maximum number of documents to process
        max_concurrent: Maximum number of documents processing/pending at once
        check_interval: How often to check status when waiting (seconds)
        mapping_file: CSV file to store the mapping
        
    Returns:
        List of tuples (row_index, original_id, datastore_document_id)
    """
    
    print(f"Using dataset with {len(dataset)} documents")
    
    mappings = []
    processed_count = 0
    start_time = time.time()
    
    # Initialize or append to mapping file
    file_exists = os.path.exists(mapping_file)
    
    with open(mapping_file, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        
        # Write header if file is new
        if not file_exists:
            writer.writerow(['row_index', 'original_id', 'datastore_document_id', 'title', 'text_preview'])
        
        for row_index, row in enumerate(dataset):
            if processed_count >= max_documents:
                break
                
            try:
                # Extract document information
                original_id = row.get('_id', f'row_{row_index}')
                title = row.get('title', '')
                text = row.get('text', '')
                text_preview = text[:100] + "..." if len(text) > 100 else text
                
                # Wait for an available slot before processing
                print(f"\nPreparing to process row {row_index} (ID: {original_id})")
                wait_for_available_slot(client, datastore_id, max_concurrent, check_interval)
                
                # Create a temporary text file with the document content using the document ID
                # Sanitize the document ID for filename use
                safe_doc_id = str(original_id).replace('/', '_').replace('\\', '_')
                temp_filename = f"doc_{safe_doc_id}.txt"
                temp_file_path = os.path.join(tempfile.gettempdir(), temp_filename)
                
                with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
                    # Write document metadata and content
                    temp_file.write(f"Original ID: {original_id}\n")
                    temp_file.write(f"Title: {title}\n")
                    temp_file.write(f"Row Index: {row_index}\n")
                    temp_file.write("---\n")
                    temp_file.write(text)
                
                try:
                    # Ingest the document
                    with open(temp_file_path, 'rb') as f:
                        ingestion_result = client.datastores.documents.ingest(datastore_id, file=f)
                        datastore_document_id = ingestion_result.id
                    
                    # Record the mapping
                    mapping_entry = (row_index, original_id, datastore_document_id)
                    mappings.append(mapping_entry)
                    
                    # Write to CSV
                    writer.writerow([row_index, original_id, datastore_document_id, title, text_preview])
                    
                    processed_count += 1
                    elapsed_time = (time.time() - start_time) / 60  # in minutes
                    rate = processed_count / elapsed_time if elapsed_time > 0 else 0
                    
                    print(f"✓ Row {row_index}: Original ID {original_id} -> Datastore ID {datastore_document_id}")
                    print(f"   Processing rate: {rate:.1f} documents/minute")
                    
                    # Show current status every few documents
                    if processed_count % 3 == 0:
                        analyze_document_status(client, datastore_id)
                    
                finally:
                    # Clean up temporary file
                    if os.path.exists(temp_file_path):
                        os.unlink(temp_file_path)
                        
            except Exception as e:
                print(f"✗ Row {row_index}: Processing error - {e}")
                continue
    
    print(f"\nProcessed {processed_count} documents successfully")
    print(f"Mapping saved to: {mapping_file}")
    
    # Final status check
    print(f"\nFinal status check:")
    analyze_document_status(client, datastore_id)
    
    return mappings

def load_hf_mapping_from_file(mapping_file: str = "hf_document_mapping.csv") -> Dict[int, Dict[str, str]]:
    """
    Load the row index to document ID mapping from CSV file.
    
    Returns:
        Dictionary mapping row_index to document info
    """
    mapping = {}
    
    if not os.path.exists(mapping_file):
        print(f"Mapping file {mapping_file} not found")
        return mapping
    
    with open(mapping_file, 'r', newline='') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            row_idx = int(row['row_index'])
            mapping[row_idx] = {
                'original_id': row['original_id'],
                'datastore_document_id': row['datastore_document_id'],
                'title': row['title'],
                'text_preview': row['text_preview']
            }
    
    return mapping

def find_document_by_row_index(row_index: int, mapping_file: str = "hf_document_mapping.csv") -> Dict[str, str]:
    """
    Find document information by row index.
    """
    mapping = load_hf_mapping_from_file(mapping_file)
    return mapping.get(row_index, {})

def find_document_by_original_id(original_id: str, mapping_file: str = "hf_document_mapping.csv") -> Dict[str, str]:
    """
    Find document information by original _id field.
    """
    mapping = load_hf_mapping_from_file(mapping_file)
    for row_idx, doc_info in mapping.items():
        if doc_info['original_id'] == original_id:
            return {**doc_info, 'row_index': row_idx}
    return {}

In [73]:
from contextual import ContextualAI
client = ContextualAI(api_key="key-bl-VPQtBsnkL2v0Gp0g9UZth2xs6tLl2Q06QQwuTKPuzyJRu8")

In [ ]:
# Assuming you've already run:
# from datasets import load_dataset
# ds = load_dataset("BeIR/fiqa", "corpus")

# CLIENT = your_client_instance
DATASTORE_ID = "7363c35f-8177-4182-b4f8-0daf6224251f"

print("Hugging Face Dataset Ingestion Script")
print("=" * 40)

# Uncomment and modify these lines when you have your client setup:

# Use the already-loaded dataset
mappings = ingest_hf_dataset_with_tracking(
    dataset=ds['corpus'],  # or just ds if it's not a DatasetDict
    client=client,
    datastore_id=DATASTORE_ID,
    max_documents=60000,
    max_concurrent=50,
    check_interval=20
)

# Test retrieval
print("\nTesting mapping retrieval:")
# By row index
doc_info = find_document_by_row_index(42)
if doc_info:
    print(f"Row 42: {doc_info}")

# By original ID
doc_info = find_document_by_original_id('382384')
if doc_info:
    print(f"ID 382384: {doc_info}")


# For now, just show what the mapping file would look like
print("This script will create a CSV mapping file with columns:")
print("- row_index: Position in the Hugging Face dataset")
print("- original_id: The '_id' field from the dataset")
print("- datastore_document_id: The ID returned by the datastore")
print("- title: Document title")
print("- text_preview: First 100 characters of text")

In [ ]:
!head hf_document_mapping.csv

I ran about 58k documents (basically text chunks) so a total of 44MB in 895 minutes or 14.9 hours - around 60ish documents per minute. I had my settings to limit it to no more than 50 concurrent document uploads at one time. (If you need more let us know)